In [2]:
from importlib.metadata import version


print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.3.0+cpu
tiktoken version: 0.7.0


### Coding an LLM architecture

- Models like GPT,Gemma,Llama, generate words sequentially and are based on decoder part of original transformer architecture


<img src="figures/03.png" width="800px">

- The next notebook will show how to load pretrained weights into our implementation, which will be compatible with model sizes of 345, 762, and 1542 million parameters
- Models like Llama and others are very similar to this model, since they are all based on the same core concepts

<img src="figures/06.png" width="600">

In [3]:
import torch.nn as nn
from supplementary import TransformerBlock, LayerNorm
import torch


class GPTModel(nn.Module):
    def __init__(self,cfg):
        super().__init__()

        self.tok_embed = nn.Embedding(cfg['vocab_size'], cfg['emb_dim'])
        self.pos_embed = nn.Embedding(cfg['context_length'],cfg['emb_dim']) 

        self.drop_emb = nn.Dropout(cfg['drop_rate'])

        self.transformer_blocks = nn.Sequential(*[TransformerBlock(cfg) for _ in range(cfg['n_layers'])])

        self.final_norm = LayerNorm(cfg['emb_dim'])
        self.out_head = nn.Linear(cfg['emb_dim'], cfg['vocab_size'], bias=False)

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_embed(in_idx)
        pos_embeds = self.pos_embed(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds  # Shape [batch_size, num_tokens, emb_size]
        x = self.drop_emb(x)
        x = self.transformer_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits


Summary of Tensor Shapes:

Input: (4, 128) - Indices of tokens in the batch.

Token Embedding Output: (4, 128, 768) - Dense token embeddings.

Positional Embedding Added: (4, 128, 768) - Combined embeddings with positional information.

Dropout Applied: (4, 128, 768) - Embeddings with dropout.

Transformer Blocks Output: (4, 128, 768) - Contextualized embeddings after multiple Transformer 
layers.

Final LayerNorm Output: (4, 128, 768) - Normalized embeddings.

Output Head (Logits): (4, 128, 50000) - Predicted logits for each token position.



In [4]:
import tiktoken

tokenizer = tiktoken.get_encoding('gpt2')

batch = []

txt1 = 'Hello I am Boy'
txt2 = "Hello I am Man"

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack((batch[0] ,batch[1]))
print(batch)

tensor([[15496,   314,   716,  6387],
        [15496,   314,   716,  1869]])


In [5]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.0,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

In [6]:
torch.manual_seed(42)

model = GPTModel(GPT_CONFIG_124M)


In [7]:
import sys

sys.getsizeof(model) # in bytes

48

In [8]:
out = model(batch)

In [9]:
print(out.shape)

torch.Size([2, 4, 50257])


In [10]:
probas = torch.softmax(out, dim=-1)  # (batch, vocab_size)

In [11]:
idx_next = torch.argmax(probas, dim=-1, keepdim=True)  # (batch, 1)

In [12]:
print(idx_next)

tensor([[[44920],
         [ 8426],
         [19840],
         [21571]],

        [[44920],
         [ 8426],
         [19840],
         [ 2382]]])


In [13]:
((idx_next.squeeze().tolist()))

[[44920, 8426, 19840, 21571], [44920, 8426, 19840, 2382]]

In [14]:
tokenizer.decode((idx_next.squeeze().tolist())[0])

' Colossusotted Graphics graduation'

In [15]:
def generate_text_simple(model, idx, max_new_tokens, context_size):
    # idx is (batch, n_tokens) array of indices in the current context
    for _ in range(max_new_tokens):
        
        # Crop current context if it exceeds the supported context size
        # E.g., if LLM supports only 5 tokens, and the context size is 10
        # then only the last 5 tokens are used as context
        idx_cond = idx[:, -context_size:]
        
        # Get the predictions
        with torch.no_grad():
            logits = model(idx_cond)
        
        # Focus only on the last time step
        # (batch, n_tokens, vocab_size) becomes (batch, vocab_size)
        logits = logits[:, -1, :]  

        # Apply softmax to get probabilities
        probas = torch.softmax(logits, dim=-1)  # (batch, vocab_size)

        # Get the idx of the vocab entry with the highest probability value
        idx_next = torch.argmax(probas, dim=-1, keepdim=True)  # (batch, 1)

        # Append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch, n_tokens+1)

    return idx


In [77]:
start_context = "hello , I am "
def generate_text_1(input_text):


    encoded = tokenizer.encode(input_text)
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) 

   


    out = generate_text_simple(
        model=model,
        idx=encoded_tensor, 
        max_new_tokens=1, 
        context_size= GPT_CONFIG_124M["context_length"]
    )


    return print(tokenizer.decode(out[0].tolist()))

# print("Output:", out)
# print("Output length:", len(out[0]))
# print(tokenizer.decode(out[0].tolist()))

In [63]:
def text_to_token_id(text,tokenizer):
    encoded= tokenizer.encode(text,allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0)
    return encoded_tensor

def token_id_to_text(id,tokenizer):
    flat_id = id.squeeze(0).tolist()
    decoded_text = tokenizer.decode(flat_id)
    return decoded_text

In [64]:
test = [1,2,3,5]
print(torch.tensor(test).shape)

torch.Size([4])


In [72]:
def generate_and_print_sample(model,tokenizer,device,start_context):

    model.eval()
    context_size = model.pos_embed.weight.shape[0]
    encoded = text_to_token_id(start_context,tokenizer).to(device)
    with torch.no_grad():
        token_id = generate_text_simple(model,encoded,max_new_tokens=5,context_size=context_size)
        
        decoded_text = token_id_to_text(token_id,tokenizer)

        print(decoded_text.replace("\n"," "))
    
    model.train()

 

In [78]:
generate_text_1(input_text ="hi how are ")

hi how are  Cabinet


In [80]:
generate_and_print_sample(model=model,tokenizer=tiktoken.get_encoding('gpt2'),device='cpu',start_context='hi how are ')

hi how are  Cabinet crib Nadbuyricted
